In [ ]:
wandb_api_key = "" #@param {type:"string"}
assert wandb_api_key != "", "It is important to track training progress with WANDB"
from huggingface_hub.hf_api import HfApi
import os
os.environ['WANDB_API_KEY'] = wandb_api_key
hfapi = HfApi()
token = ''
hfapi.set_access_token(token)
!mkdir /root/.huggingface -p

with open("/root/.huggingface/token", "w+") as text_file:
    text_file.write(token)


In [ ]:
!accelerate config

# Train model

In [ ]:
hf_NFT_collections = sorted( [datainfo.id for datainfo in hfapi.list_datasets(author="huggingnft") \
                           if datainfo.id.split('/')[-1] ] )

collectionA = ''
collectionB = ''
assert collectionA in hf_NFT_collections and collectionB in hf_NFT_collections, f'Error, collectionA and collectionB must be one of {hf_NFT_collections}'
# root destination to which the trained models will be pushed to 
DEST_ORGANIZATION = ...
hyperparams_cyclegan = { 
                'lr':[0.0002], 
                'num_epochs':[200], 
                'decay_epoch':[80] ,
                'n_residual_blocks':[9], 
                'lambda_cyc':[10.0], 
                'lambda_id':[5.0], 
                'beta1':[0.5], 'beta2':[0.999], 
                'batch_size':[8], 
                'source_dataset_name':[f'huggingnft/{collectionA}'],
                'target_dataset_name':[f'huggingnft/{collectionB}'],
                'channels':[3], 
                'checkpoint_interval':[5], 
                #'cpu':[False], 
                'epoch':[0], 
                #'fp16':[False], 
                'image_size':[256],
                'mixed_precision':['no'], 
                'num_workers':[8], 
                'organization_name':[DEST_ORGANIZATION], 
                'wandb':[True],
                'push_to_hub':[True], 
                'sample_interval':[10] }

call_params = ['accelerate', 'launch',
                       '--config_file','~/.cache/huggingface/accelerate/default_config.yaml',
                       'train.py']
for idx, grid in enumerate(ParameterGrid(hyperparams_cyclegan)):
        grid['output_dir'] = 'experiments'

        
for k,v in grid.items():

            if k in  ['cpu', 'wandb','fp16','push_to_hub']:
                if v is True:
                    call_params.append(f'--{k}' )
            else:
                call_params.append(f'--{k}' )
                call_params.append(f'{v}' )

        print(' '.join(call_params))
        call(call_params)

# Train all possible translation models

In [ ]:
from sklearn.model_selection import ParameterGrid
from subprocess import call
import os
import json
from huggingface_hub.hf_api import HfApi
import copy


hfapi = HfApi()

# root destination to which the trained models will be pushed to 
DEST_ORGANIZATION = 'Chris1' #'huggingnft'

In [ ]:
hf_NFT_collections = sorted( [datainfo.id for datainfo in hfapi.list_datasets(author="huggingnft") \
                           if datainfo.id.split('/')[-1] ] )

In [ ]:
cnt = 1
total = len(hf_NFT_collections)* (len(hf_NFT_collections) -1)//2
for idx1, hf_datapath1 in enumerate(hf_NFT_collections):
    for idx2 in range(idx1+1, len(hf_NFT_collections) )  :
        hf_datapath2 = hf_NFT_collections[idx2] 
        collectionA = hf_datapath1.split('/')[-1]
        collectionB = hf_datapath2.split('/')[-1]
  
        hyperparams_cyclegan = { 
                'lr':[0.0002], 
                'num_epochs':[200], 
                'decay_epoch':[80] ,
                'n_residual_blocks':[9], 
                'lambda_cyc':[10.0], 
                'lambda_id':[5.0], 
                'beta1':[0.5], 'beta2':[0.999], 
                'batch_size':[8], 
                'source_dataset_name':[f'huggingnft/{collectionA}'],
                'target_dataset_name':[f'huggingnft/{collectionB}'],
                'channels':[3], 
                'checkpoint_interval':[5], 
                #'cpu':[False], 
                'epoch':[0], 
                #'fp16':[False], 
                'image_size':[256],
                'mixed_precision':['no'], 
                'num_workers':[8], 
                'organization_name':[DEST_ORGANIZATION], 
                'wandb':[True],
                'push_to_hub':[True], 
                'sample_interval':[10] }


        call_params = ['accelerate', 'launch',
                               '--config_file','~/.cache/huggingface/accelerate/default_config.yaml',
                               'train.py']
        for idx, grid in enumerate(ParameterGrid(hyperparams_cyclegan)):
                grid['output_dir'] = 'experiments'

                for k,v in grid.items():

                    if k in  ['cpu', 'wandb','fp16','push_to_hub']:
                        if v is True:
                            call_params.append(f'--{k}' )
                    else:
                        call_params.append(f'--{k}' )
                        call_params.append(f'{v}' )

                print(' '.join(call_params))
                call(call_params)
   
        
        
        
        
        
        
        
        cnt += 1